## Deliverable 2. Create a Customer Travel Destinations Map

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Valdivia,-39.8142,-73.2459,75.20,41,0,6.91,CL,2021-01-13 20:12:02
1,1,Bredasdorp,-34.5322,20.0403,69.80,78,100,16.11,ZA,2021-01-13 20:15:57
2,2,Castro,-24.7911,-50.0119,74.14,86,51,9.84,BR,2021-01-13 20:15:57
3,3,Leningradskiy,69.3833,178.4167,-13.58,88,100,8.16,RU,2021-01-13 20:15:57
4,4,Khasan,42.4308,130.6434,16.36,83,0,14.99,RU,2021-01-13 20:15:57


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Valdivia,-39.8142,-73.2459,75.20,41,0,6.91,CL,2021-01-13 20:12:02
7,7,Quatre Cocos,-20.2078,57.7625,78.01,94,75,2.30,MU,2021-01-13 20:15:57
8,8,Rikitea,-23.1203,-134.9692,77.59,78,33,15.79,PF,2021-01-13 20:15:57
14,14,Itarema,-2.9248,-39.9167,82.24,72,84,14.43,BR,2021-01-13 20:15:58
16,16,Padang,-0.9492,100.3543,77.92,71,93,4.32,ID,2021-01-13 20:15:58
...,...,...,...,...,...,...,...,...,...,...
688,688,Calabasas,34.1578,-118.6384,78.01,17,1,4.61,US,2021-01-13 20:16:50
689,689,Tete,-16.1564,33.5867,77.00,100,75,4.79,MZ,2021-01-13 20:16:50
691,691,Bowen,-20.0167,148.2333,75.20,88,100,11.50,AU,2021-01-13 20:16:50
694,694,Port Blair,11.6667,92.7500,77.07,77,0,10.71,IN,2021-01-13 20:16:51


In [6]:
# 4a. Determine if there are any empty rows
preferred_cities_df.count()

City_ID       198
City          198
Lat           198
Lng           198
Max Temp      198
Humidity      198
Cloudiness    198
Wind Speed    198
Country       198
Date          198
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Valdivia,CL,75.20,-39.8142,-73.2459,
7,Quatre Cocos,MU,78.01,-20.2078,57.7625,
8,Rikitea,PF,77.59,-23.1203,-134.9692,
14,Itarema,BR,82.24,-2.9248,-39.9167,
16,Padang,ID,77.92,-0.9492,100.3543,
17,Tiarei,PF,84.20,-17.5333,-149.3333,
20,Avarua,CK,84.20,-21.2078,-159.7750,
26,Touros,BR,82.40,-5.1989,-35.4608,
28,Atuona,PF,77.65,-9.8000,-139.0333,
32,Puerto Ayora,EC,87.80,-0.7393,-90.3518,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the coordinates to a location key for the params dictionary
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping.")

Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.
Hotel not found... Skipping.


In [16]:
# Print dataframe
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Valdivia,CL,75.20,-39.8142,-73.2459,
7,Quatre Cocos,MU,78.01,-20.2078,57.7625,Emeraude Beach Attitude Hotel
8,Rikitea,PF,77.59,-23.1203,-134.9692,Pension Maro'i
14,Itarema,BR,82.24,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
16,Padang,ID,77.92,-0.9492,100.3543,
...,...,...,...,...,...,...
688,Calabasas,US,78.01,34.1578,-118.6384,The Anza A Calabasas Hotel
689,Tete,MZ,77.00,-16.1564,33.5867,Zambeze Paraíso Misterioso
691,Bowen,AU,75.20,-20.0167,148.2333,Castle Motor Lodge Motel
694,Port Blair,IN,77.07,11.6667,92.7500,Welcomhotel Bay Island Port Blair


In [21]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=["Hotel Name"])
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Valdivia,CL,75.20,-39.8142,-73.2459,
7,Quatre Cocos,MU,78.01,-20.2078,57.7625,Emeraude Beach Attitude Hotel
8,Rikitea,PF,77.59,-23.1203,-134.9692,Pension Maro'i
14,Itarema,BR,82.24,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
16,Padang,ID,77.92,-0.9492,100.3543,
...,...,...,...,...,...,...
688,Calabasas,US,78.01,34.1578,-118.6384,The Anza A Calabasas Hotel
689,Tete,MZ,77.00,-16.1564,33.5867,Zambeze Paraíso Misterioso
691,Bowen,AU,75.20,-20.0167,148.2333,Castle Motor Lodge Motel
694,Port Blair,IN,77.07,11.6667,92.7500,Welcomhotel Bay Island Port Blair


In [ ]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
